In [14]:
from multiprocessing import Process,Queue
import os

INF = 1e9
SourceNode = 1
class HyperNode:
    def __init__(self,n_id):
        self.n_id = n_id
        self.dist = INF 
        self.edges = []
        
    def AddEdge(self,edge):
        self.edges.append(edge)
        
    def GetHyperEdges(self):
        return self.edges
    
    def GetValue(self):
        return self.dist
    
                    
class HyperEdge:
    def __init__(self,e_id):
        self.e_id = e_id
        self.nodes = []
        
    def AddNode(self,node):
        self.nodes.append(node)
    
    def GetHyperNodes(self):
        return self.nodes
    
    def getValue(self):
        return 1
    
# def solve():
#     while 
#         for n_id,node in Node.items():
#             node.compute(msg[n_id])
        


# def load_data():
#     with open("./test_data/partition_method.txt") as file :
#         for line in file:
#             v_id,par_id = line[0:-1].split(" ")
#             Part[v_id] = par_id
    
#     with open("./test_data/github_vertex_info.txt") as file:
#         for line in file:
#             data = line[0:-1].split(" ")
#             v_id = int(data[0])
#             Node[v_id] = HyperNode(v_id)
#             for e_id in data[1:]:
#                 Node[v_id].AddEdge(int(e_id))
                
#     with open("./test_data/github_edge_info.txt") as file:
#         for line in file:
#             data = line[0:-1].split(" ")
#             e_id = int(data[0])
#             Edge[e_id] = HyperEdge(e_id)
#             for n_id in data[1:]:
#                 Edge[e_id].AddNode(int(n_id))

        
    

In [16]:
class Master:
    def __init__(self,p):
        self.p = p
        self.Node = {}
        self.Edge = {}
        self.Part = {}
        self.Worker = []
        for i in range(p) :
            self.Worker.append(Worker(i))

    def load_data(self):
        with open("./test_data/partition_method.txt") as file :  # (n_id,p_id)
            for line in file:
                v_id,par_id = line[0:-1].split(" ")
                v_id,par_id = int(v_id),int(par_id)
                self.Part[v_id] = par_id
                    
        with open("./test_data/github_vertex_info.txt") as file:  # v_id, e1_id, e2_id, ...
            for line in file:
                data = line[0:-1].split(" ")
                v_id = int(data[0])
                self.Node[v_id] = HyperNode(v_id)
                for e_id in data[1:]:
                    self.Node[v_id].AddEdge(int(e_id))
                    
        with open("./test_data/github_edge_info.txt") as file:   # e_id, v1_id, v2_id,...
            for line in file:
                data = line[0:-1].split(" ")
                e_id = int(data[0])
                self.Edge[e_id] = HyperEdge(e_id)
                for n_id in data[1:]:
                    self.Edge[e_id].AddNode(int(n_id))

                    
    def distribute_node(self,node,w_id):
        if node.n_id in self.Worker[w_id].Node : return # nonsence
        self.Worker[w_id].receive_node(node)
#         self.Worker[w_id].Node[n_id] = self.Node[n_id]
    
    def distribute_edge(self,edge,w_id):
        if edge.e_id in self.Worker[w_id].Edge : return # avoid repeat send
#         self.Worker[w_id].Edge[e_id] = self.Edge[e_id]
        self.Worker[w_id].receive_edge(edge)
    
    def distribute_part(self):
        for worker in self.Worker:
            worker.receive_part(self.Part)
        
    def distribute(self):
        self.distribute_part()
                
        for n_id,node in self.Node.items():
            self.distribute_node(node,self.Part[n_id])
            for e_id in node.edges:
                self.distribute_edge(self.Edge[e_id],self.Part[n_id])
                
    def compute(self):
        turn = 0
        msg_cnt = 1
        while msg_cnt != 0 :
            turn += 1
            msg_cnt = 0
            result = Queue()
            
            thread_pool = []
            for worker in self.Worker:
#                 print("start worker!")
                thread_pool.append(Process(target= worker.compute,args=(result,)))
                thread_pool[-1].start()
            
            for thread in thread_pool:
#                 print("finished worker!")
                thread.join()
            
            msg_cnt = sum([result.get() for i in thread_pool])            
            print(turn," msg_cnt:",msg_cnt)
            print("compute finished!")
            
            thread_pool = []
            for worker in self.Worker:
#                 worker.communicate_send()
                thread_pool.append(Process(target= worker.communicate_send()))
                thread_pool[-1].start()
            
            for thread in thread_pool:
                thread.join()
            print("comunicate send finished!")
            
            thread_pool = []
            for worker in self.Worker:   
#                 worker.communicate_rece()
                thread_pool.append(Process(target= worker.communicate_rece()))
                thread_pool[-1].start()
            
            for thread in thread_pool:
                thread.join()
            print("communicate rece finished!")
            

    
    def init(self):
        self.Worker[self.Part[SourceNode ]].msg[SourceNode ] = [0]
            
            
                    
class Worker:
    def __init__(self,w_id):
        self.w_id = w_id
        self.Node = {}
        self.Edge = {}
        self.Part = {}
        self.msg = {}
        self.sendBuf = []
        self.recvBuf = []
        filepath = "./pipeline/"+str(self.w_id)
        open(filepath,"w")
        
    def cal_node(self,node):
        msgs = self.msg[node.n_id]
        mindist = INF 
        mindist = min(msgs)
        sendMessage = []
        if mindist < node.dist : 
            node.dist = mindist
            for e_id in node.GetHyperEdges():
                hyperEdge = self.Edge[e_id]
                for n_id in hyperEdge.GetHyperNodes():
                    sendMessage.append((n_id,mindist+hyperEdge.getValue()))
#                     SendMessageTo(self.v_id,target,mindist+hyperEdge.getValue())
        return sendMessage
    
    def receive_node(self,Node):
        self.Node[Node.n_id] = Node
        
    def receive_edge(self,Edge):
        self.Edge[Edge.e_id] = Edge
        
    def receive_part(self,Part):
        self.Part = Part
    
    def compute(self,queue):
        for tar,value in self.recvBuf: # receive msg from previous superstep
            if tar not in self.msg :
                self.msg[tar] = []
            self.msg[tar].append(value)
        self.recvBuf = []
        
        msg_cnt = 0
        for n_id,node in self.Node.items():
            if n_id not in self.msg : continue
            msg = self.cal_node(node) # compute and get msg info
            msg_cnt += len(msg)
            for tar,value in msg:  # msg transfer judge
                if self.Part[tar] == self.w_id:
                    self.recvBuf.append((tar,value))
                else : 
                    self.sendBuf.append((tar,value))
        queue.put(msg_cnt)
    
    def communicate_send(self):
        filepath = "./pipeline/"
        for tar,value in self.sendBuf:
#             print(self.Part[tar]," ",value)
            with open(filepath+str(self.Part[tar]),"a") as file:
                file.write(str(tar)+" "+str(value)+"\n")
                
    def communicate_rece(self):
        filepath = "./pipeline/"+str(self.w_id)
        with open(filepath,"r") as file:
            for line in file:
                tar,value =  line[0:-1].split(" ")
                tar,value = int(tar),int(value)
                self.recvBuf.append((tar,value))
                
        open(filepath,"w")
            
            
        
    
master = Master(4)
master.load_data()
master.distribute()
master.init()
master.compute()
for worker in master.Worker:
    print(len(worker.Part))
    print(len(worker.Node))

start worker!
start worker!
start worker!
start worker!
finished worker!
finished worker!
finished worker!
finished worker!
1  msg_cnt: 1017
compute finished!
comunicate send finished!
communicate rece finished!
start worker!
start worker!
start worker!
start worker!
finished worker!
finished worker!
finished worker!
finished worker!
2  msg_cnt: 1017
compute finished!
comunicate send finished!
communicate rece finished!
start worker!
start worker!
start worker!
start worker!
finished worker!
finished worker!
finished worker!
finished worker!
3  msg_cnt: 1017
compute finished!
comunicate send finished!
communicate rece finished!
start worker!
start worker!
start worker!
start worker!
finished worker!
finished worker!
finished worker!
finished worker!
4  msg_cnt: 1017
compute finished!
comunicate send finished!
communicate rece finished!
start worker!
start worker!
start worker!
start worker!
finished worker!
finished worker!
finished worker!
finished worker!
5  msg_cnt: 1017
compute fin

comunicate send finished!
communicate rece finished!
start worker!
start worker!
start worker!
start worker!
finished worker!
finished worker!
finished worker!
finished worker!
40  msg_cnt: 1017
compute finished!
comunicate send finished!
communicate rece finished!
start worker!
start worker!
start worker!
start worker!
finished worker!
finished worker!
finished worker!
finished worker!
41  msg_cnt: 1017
compute finished!
comunicate send finished!
communicate rece finished!
start worker!
start worker!
start worker!
start worker!
finished worker!
finished worker!
finished worker!
finished worker!
42  msg_cnt: 1017
compute finished!
comunicate send finished!
communicate rece finished!
start worker!
start worker!
start worker!
start worker!
finished worker!
finished worker!
finished worker!
finished worker!
43  msg_cnt: 1017
compute finished!
comunicate send finished!
communicate rece finished!
start worker!
start worker!
start worker!
start worker!
finished worker!
finished worker!
finis

KeyboardInterrupt: 

In [17]:
for worker in master.Worker:
    print(len(worker.Edge))

78935
17019
15782
20493
